In [ ]:
import speech_recognition as sr
r = sr.Recognizer()
with sr.Microphone() as source:
    print("Speak Anything :")
    audio = r.listen(source)
    try: 
        text = r.recognize_google(audio)
        print("You said : {}".format(text))
    except Exception as e:
        print("Sorry could not recognize what you said")
 

In [ ]:
pip install SpeechRecognition


In [ ]:
sudo apt-get update
sudo apt-get install portaudio19-dev


In [ ]:
sudo apt-get update
sudo apt-get install portaudio19-dev


In [ ]:
pip install pyaudio


In [12]:
pip install numpy pandas librosa scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import speech_recognition as sr
import re
from sklearn.utils.class_weight import compute_class_weight


In [24]:

file_path = '/home/amrtutha/Downloads/ML task 3(4)/dataset/metadata.csv'
data = pd.read_csv(file_path)


In [25]:

def transcribe_audio(file_path):
    recognizer = sr.Recognizer()
    try:
        with sr.AudioFile(file_path) as source:
            audio = recognizer.record(source)
        return recognizer.recognize_google(audio)
    except sr.UnknownValueError:
        return ""
    except sr.RequestError:
        return ""


In [26]:

print(data.head())


                                                path           speaker  \
0  dataset/audio/speakers/2BqVo8kVB2Skwgyb/0a3129...  2BqVo8kVB2Skwgyb   
1  dataset/audio/speakers/2BqVo8kVB2Skwgyb/0ee42a...  2BqVo8kVB2Skwgyb   
2  dataset/audio/speakers/2BqVo8kVB2Skwgyb/1d9f39...  2BqVo8kVB2Skwgyb   
3  dataset/audio/speakers/2BqVo8kVB2Skwgyb/269fc2...  2BqVo8kVB2Skwgyb   
4  dataset/audio/speakers/2BqVo8kVB2Skwgyb/5bbda3...  2BqVo8kVB2Skwgyb   

            action  object fluency_level         current_language  gender  \
0  change language    none      advanced  English (United States)  female   
1         activate   music      advanced  English (United States)  female   
2       deactivate  lights      advanced  English (United States)  female   
3         increase  volume      advanced  English (United States)  female   
4         increase  volume      advanced  English (United States)  female   

  age_range  
0     22-40  
1     22-40  
2     22-40  
3     22-40  
4     22-40  


In [27]:

def preprocess_text(text):
    text = text.lower()  
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'\d+', '', text)  
    text = re.sub(r'[^\w\s]', '', text) 
    return text


In [28]:
texts = data['path']
labels = data['action']
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)


In [29]:
 
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2)) 
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [30]:

class_weights = compute_class_weight('balanced', classes=labels.unique(), y=labels)
class_weight_dict = dict(zip(labels.unique(), class_weights))


In [31]:

model = MultinomialNB(class_prior=[class_weight_dict.get(label, 1) for label in labels.unique()])
model.fit(X_train_tfidf, y_train)


MultinomialNB(class_prior=[1.7707097933513027, 2.4915297092288244,
                           3.5702898550724638, 0.5155113784985613,
                           0.5512727272727272])

In [22]:
 
y_pred = model.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=1) 


In [23]:

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:\n', report)


Accuracy: 0.42
Classification Report:
                  precision    recall  f1-score   support

       activate       0.13      0.02      0.03       162
change language       0.32      0.78      0.45       225
     deactivate       0.14      0.11      0.12       118
       decrease       0.48      0.24      0.32       690
       increase       0.49      0.60      0.54       776

       accuracy                           0.42      1971
      macro avg       0.31      0.35      0.29      1971
   weighted avg       0.41      0.42      0.38      1971

